# **Instalamos las dependencias y librerias requeridas para nuestro notebook**

En primera instancia, vamos a probar a cargar modelos de la libreria de hugging-face para probar, en una menor escala, el rendimiento de modelos LLM ya preentrenados previamente, con el fin de evaluar si esta respondiendo correctamente a los promts que el usuario envía.

In [ ]:
!pip install transformers datasets
!pip install --upgrade datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import pipeline

# **Cargar modelos**

Cargamos el modelo t5-large que es un modelo pre-entrenado proveniente de la libreria de Hugging Face. Hugging Face es una empresa que proporciona una plataforma para modelos de procesamiento de lenguaje natural (NLP), conocida por su colección de modelos pre-entrenados y herramientas de fácil uso para el desarrollo de NLP, accesible para todo el mundo.

El modelo '**t5-large**' es una modelo "Text-to-Text Transfer Transformer", es una arquitectura de modelo de lenguaje desarrollada por Google Research. Fue presentado en un artículo titulado "Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer". La idea detrás de T5 es tratar todas las tareas de NLP como un problema de transformación de texto en texto. Por ejemplo, la traducción se trata como la conversión del texto de un idioma a otro, y la clasificación se trata como la conversión de un texto en una etiqueta.

In [ ]:
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Se crea una instancia del tokenizador para el modelo T5. El método from_pretrained() carga un tokenizador que ha sido pre-entrenado con el modelo 't5-large'. Este tokenizador convertirá el texto de entrada en el formato que el modelo T5 espera.

De manera similar al tokenizador, esta línea crea una instancia del modelo T5 utilizando el método from_pretrained(), cargando el modelo 't5-large' pre-entrenado. Este modelo lo vamos a utilizar para generacion de texto, dado un prompt.

**Generamos una pregunta basica**

In [ ]:
def generate_answer(question):
    # Preparación del prompt de entrada sin repetir la estructura de la pregunta en la parte de la respuesta.
    input_text = f"question: {question} </s>"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generación de texto con muestreo activado y parámetros ajustados.
    output = model.generate(input_ids, max_new_tokens=50, no_repeat_ngram_size=2,
                            temperature=0.9, top_p=0.95, do_sample=True)

    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

question = "What is the capital of France?"
answer = generate_answer(question)
print("Respuesta generada:", answer)


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:309: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Respuesta generada: Paris


La respuesta obtenida a nuestra pregunta, es correcta.

**Evaluación de los modelos**

A continuacion vamos a evaluar que tan bien ha respondido nuestro modelo a nuestra pegunta

In [ ]:
nli_model_name = 'roberta-large-mnli'
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluamos si una premisa y una hipótesis están en contradicción, son neutrales o están en implicación (entailment) utilizando el modelo RoBERTa entrenado en el conjunto de datos Multi-Genre Natural Language Inference (MNLI), procedente tambien de Hugging Face.

Creamos una funcion encargada de evaluar la respuesta obtenida del modelo t5-large

In [ ]:
def evaluate_nli(premise, hypothesis):
    # Preprocesar la entrada para el modelo NLI
    inputs = nli_tokenizer.encode_plus(premise, hypothesis, return_tensors='pt', max_length=512, truncation=True, padding=True)
    # Realizar la predicción
    with torch.no_grad():
        outputs = nli_model(**inputs)
    # Calcular las probabilidades
    probs = torch.softmax(outputs.logits, dim=1)
    # Asignar las probabilidades a las etiquetas correspondientes
    entailment, contradiction, neutral = probs.squeeze().tolist()

    return {'entailment': entailment, 'contradiction': contradiction, 'neutral': neutral}

In [ ]:
premise = question
hypothesis = answer

# Evaluar la respuesta
nli_result = evaluate_nli(premise, hypothesis)
print(f"Resultados NLI: {nli_result}")

Resultados NLI: {'entailment': 0.0528288409113884, 'contradiction': 0.7834100723266602, 'neutral': 0.16376104950904846}


Probamos a reformular la pregunta y la respuesta

In [ ]:
premise = "The question is about the capital of France."
hypothesis = "The capital of France is Paris."

nli_result = evaluate_nli(premise, hypothesis)
print(f"Resultados NLI ajustados: {nli_result}")

Resultados NLI ajustados: {'entailment': 0.004765156656503677, 'contradiction': 0.9918850660324097, 'neutral': 0.0033498266711831093}


In [ ]:
# Otro intento con una formulación ligeramente diferente
premise = "What is being asked about is the capital city of a European country, France."
hypothesis = "Paris is the capital city of France."

nli_result = evaluate_nli(premise, hypothesis)
print(f"Resultados NLI con reformulación: {nli_result}")


Resultados NLI con reformulación: {'entailment': 0.002543726237490773, 'contradiction': 0.9760329723358154, 'neutral': 0.021423300728201866}


Los resultados de NLI que obtienes, con una alta probabilidad de contradicción, son inesperados dado el claro caso de entailment entre la premisa y la hipótesis. Esto sugiere que podría haber aspectos en la forma en que el modelo de NLI está interpretando la relación entre tu premisa y tu hipótesis que no coinciden con la evaluación humana lógica.

Observamos un sesgo en la evaluacion de las respuestas, ya que para una pregunta concreta obtenemos una respuesta correcta y valida, pero el benchmark nos indica que la respuesta no esta siendo valida, cuando la lógica humana nos dice que si

**Prueba con otros modelos y otros metodos de evaluacion**

En este caso vamos a utilizar el modelo 'distilbert-base-uncased', es una versión más compacta y eficiente de BERT, diseñada para proporcionar una comprensión del lenguaje natural con menos recursos computacionales. Manteniendo gran parte de la potencia de BERT, este modelo reducido utiliza mecanismos de atención y ha sido entrenado para comprender y predecir texto. Desarrollado por Hugging Face, DistilBERT es ideal para entornos donde los recursos son limitados

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Al igual que hicimos con el modelo "t5-large", cargamos el tokenizador, para que pueda leer las entradas nuestro modelo y posteriormente, cargamos el modelo preentrenado de distilbert, procedente de Hugging Face

Creamos una funcion para codificar y las entradas de texto con el tokenizador de distilbert, se aplica truncamiento y relleno para asegurarse de que todas las entradas tengan la misma longitud

In [ ]:
def encode(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length', return_token_type_ids=False)

Para la evaluacion de este modelo, utilizaremos el concepto de ***benchmark***, en este contexto, consiste en un conjunto de datos de prueba y una serie de métricas que permiten medir de manera objetiva aspectos como la precisión, la eficiencia y la capacidad de comprensión del lenguaje de los modelos

In [ ]:
# Cargar y codificar el dataset
dataset = load_dataset('glue', 'mrpc')
encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

En este caso el benchmark utilizado es ***GLUE***, que es una colección de varios conjuntos de datos de NLP con el objetivo de evaluar el rendimiento de los modelos en una variedad de tareas de comprensión del lenguaje. ***MRPC*** (Microsoft Research Paraphrase Corpus) es uno de esos conjuntos de datos dentro de GLUE y se centra en la tarea de identificar si dos frases son semánticamente equivalentes, es decir, si una frase es una paráfrasis de la otra.

Creamos un DataLoader para el conjunto de evaluación y convertimos el conjunto de datos en un DataFrame de pandas

In [ ]:
val_loader = DataLoader(encoded_dataset['validation'], batch_size=16)
validation_df = pd.DataFrame(dataset['validation'])
validation_df.head()

,sentence1,sentence2,label,idx
0,He said the foodservice pie business doesn 't ...,""" The foodservice pie business does not fit ou...",1,9
1,Magnarelli said Racicot hated the Iraqi regime...,"His wife said he was "" 100 percent behind Geor...",0,18
2,"The dollar was at 116.92 yen against the yen ,...","The dollar was at 116.78 yen JPY = , virtually...",0,25
3,The AFL-CIO is waiting until October to decide...,The AFL-CIO announced Wednesday that it will d...,1,32
4,No dates have been set for the civil or the cr...,No dates have been set for the criminal or civ...,0,33


In [ ]:
def evaluate_model(data_loader, model):
    model.eval()
    total_correct = 0
    total = 0
    for batch in data_loader:
        # Mueve los inputs a la misma device que el modelo
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'label'}

        # Predicciones del modelo
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Compara las predicciones con las etiquetas verdaderas y calcula el total correcto
        total_correct += (predictions == batch['label'].to(model.device)).sum().item()
        total += batch['label'].size(0)

    # Calcula la precisión
    accuracy = total_correct / total
    return accuracy

La función evaluate_model toma un DataLoader y el modelo, y evalúa la precisión del modelo pasando los lotes de datos a través del modelo y comparando las predicciones con las etiquetas verdaderas.

In [ ]:
accuracy = evaluate_model(val_loader, model)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.41


Obtenemos una baja precision lo que nos quiere decir que, nuestro modelo predice con un 41% si las frases de la columna 1 y la columna 2 son paráfrasis entre si.

De nuevo, no hemos obtenido una precisión adecuada para poder utilizarlo como proceso para corregir el sesgo y las alucinaciones de modelos LLM.

**Probamos a iniciar una pipeline de NLI con un modelo nuevo de hugging-face**

Para este proceso, emplearemos el uso de ***pipelines*** de la libreria transoformers.  Las pipelines encapsulan los procesos de inferencia de varios modelos de NLP y permiten aplicarlos de manera directa y sencilla para tareas comunes como clasificación, generación de texto, y en este caso, clasificación de afirmaciones.

En este caso se crea una instancia de una pipeline que está diseñada para realizar clasificación de ***zero-shot-classification***, que es una forma de clasificación donde el modelo no ha sido entrenado específicamente en las clases de etiquetas que se utilizan durante la inferencia. Esto es útil para aplicaciones donde las etiquetas pueden variar dinámicamente.
El modelo utilizado es "**facebook/bart-large-mnli**", un modelo basado en **BART** (Bidirectional and Auto-Regressive Transformers) de Facebook, adaptado y afinado en el conjunto de datos MNLI (Multi-Genre Natural Language Inference). Este modelo ha sido entrenado para comprender las relaciones semánticas entre pares de oraciones y puede determinar si una afirmación es verdadera, falsa, o neutra en relación con una hipótesis dada.

In [ ]:
nli_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Introduccimos una afirmacion, para ser evalauda por nuestro modelo

In [ ]:
afirmacion_generada = "The Eiffel Tower is in Paris"

# Evaluar la afirmación
resultado = nli_pipeline(sequences=afirmacion_generada, candidate_labels=["factual", "ficción"], hypothesis_template="Esta afirmación es {}.")
print(resultado)


{'sequence': 'The Eiffel Tower is in Paris', 'labels': ['factual', 'ficción'], 'scores': [0.8147465586662292, 0.18525342643260956]}


Las metricas en las que se mide nuestro modelo son:
- Factualidad, cuanto de verdadero es la afirmación generada
- Ficcion, cuanto de falsa es la afirmacion generada

A pesar de ser un hecho veridico, que a priori podriamos pensar que deberia ser un 100% factual, en muchas ocasiones, no obtenemos la maxima puntuación debido a no añadir un mayor contexto a la afirmación

En conclusión, este resultado muestra que el modelo ha evaluado correctamente la afirmación como un hecho verídico basándose en su entrenamiento y conocimiento. Estos resultados se pueden utilizar para automatizar la verificación de hechos o para filtrar información basada en su veracidad en aplicaciones de NLP.

# **Conclusiones**

Tras la aplicacion de tres metodos:
- Uso de T5-large para generación de respuestas y la evaluación de la calidad de estas respuestas usando RoBERTa-large-mnli.
- Uso de DistilBERT junto con la evaluacion de benchmark de GLUE y MRPC
- Uso de BART-MNLI en Clasificación Zero-Shot

Obtenemos que el mejor rendimiento de los tres metodos aplicados es el uso de BART-MNLI en Clasificación Zero-Shot, debido a su gran simplicidad en el código y al resultado en cuanto métricas se refiere.
A pesar de ello, llevar este método a una mayor escala, trabajando con datos propios de una empresa, puede llegar a ser un método costoso y que requiera un gran almacenamiento.